In [74]:
import pandas as pd
import numpy as np
import os

In [149]:
xtrain=np.load(r'C:/Users/409/Desktop/新增資料夾/all 2/X_train2.npy',mmap_mode="r")

In [150]:
xtest=np.load(r'C:/Users/409/Desktop/新增資料夾/all 2/x_test2.npy',mmap_mode="r")

In [151]:
ytrain=np.load(r'C:/Users/409/Desktop/新增資料夾/all 2/y_train2.npy', mmap_mode="r")
ytrain

memmap([2, 0, 0, ..., 0, 0, 3], dtype=int64)

In [152]:
X_train_normalize=xtrain.astype('float32')/255
X_test_normalize=xtest.astype('float32')/255

In [153]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X_train_normalize, ytrain, random_state=0, test_size=0.2)

In [154]:
from keras.utils import np_utils
print(Xtrain.shape)
print(X_test_normalize.shape)

(1744, 224, 224, 3)
(524, 224, 224, 3)


In [155]:
from keras import layers
from keras import models
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

In [156]:
model=Sequential()
model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64,kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01),activation='relu'))
model.add(layers.Dense(32,kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01),activation='relu'))
model.add(layers.Dense(4,activation='softmax'))

In [157]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 109, 109, 32)      9248      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 52, 52, 64)        18496     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 24, 24, 64)        36928     
__________

In [158]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [159]:
ytrain=to_categorical(ytrain)

In [160]:
ytrain

array([[0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]], dtype=float32)

In [192]:
model.fit(Xtrain,ytrain,epochs=10,batch_size=64)

Epoch 1/10
1744/1744 [==============================] - 2s 1ms/step - loss: 1.8020 - acc: 0.7036
Epoch 2/10
1744/1744 [==============================] - 2s 1ms/step - loss: 1.7776 - acc: 0.7076
Epoch 3/10
1744/1744 [==============================] - 2s 1ms/step - loss: 1.7990 - acc: 0.7058
Epoch 4/10
1744/1744 [==============================] - 2s 1ms/step - loss: 1.8033 - acc: 0.7070
Epoch 5/10
1744/1744 [==============================] - 2s 1ms/step - loss: 1.7773 - acc: 0.7173
Epoch 6/10
1744/1744 [==============================] - 2s 1ms/step - loss: 1.7948 - acc: 0.7081
Epoch 7/10
1744/1744 [==============================] - 2s 1ms/step - loss: 1.7593 - acc: 0.7202
Epoch 8/10
1744/1744 [==============================] - 2s 1ms/step - loss: 1.7499 - acc: 0.7213
Epoch 9/10
1744/1744 [==============================] - 2s 1ms/step - loss: 1.7353 - acc: 0.7236
Epoch 10/10
1744/1744 [==============================] - 2s 1ms/step - loss: 1.7314 - acc: 0.7357


In [193]:
#Y_pred = model.predict(X_test_normalize)
#Y_pred

In [194]:
Y_pred = model.predict(Xtest)

In [195]:
ans = [np.argmax(i) for i in Y_pred]
print(ans.count(0), ans.count(1), ans.count(2),ans.count(3))

369 0 23 44


In [196]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest, ans)

0.5825688073394495

In [73]:
def toCSV(filename, pred):
    with open(r"C:\Users\409\Desktop\sample_submission2.csv", "r") as f:
        data = f.readlines()
        data = [i.replace("\n","") for i in data]
    tmp = [str(data[i][:-1])+str(pred[i-1])+"\n" for i in range(1,len(data))]
    tmp.insert(0,data[0]+"\n")
    with open(filename, "w") as f:
        for i in tmp:
            f.write(i)
toCSV(r"C:\Users\409\Desktop\ze.csv", ans)

In [10]:
from keras.applications import VGG16

conv_base=VGG16(weights='imagenet',
                include_top=False,
                input_shape=(224,224,3))

In [11]:
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras.layers import Dense
x = conv_base.output
x = Flatten()(x)
conv_base=Model(conv_base.input,x)
num_classes=64
x=Dense(num_classes, activation='relu')(conv_base.output)
conv_base=Model(conv_base.input,x)
num_classes=4
x=Dense(num_classes, activation='softmax')(conv_base.output)
# 重新建立模型結構
conv_base=Model(conv_base.input,x)

In [12]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [13]:
conv_base.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
conv_base.fit(X_train_normalize,ytrain,epochs=10,batch_size=128)

Epoch 1/10
2180/2180 [==============================] - 20s 9ms/step - loss: 5.9519 - acc: 0.6307
Epoch 2/10
2180/2180 [==============================] - 20s 9ms/step - loss: 5.9519 - acc: 0.6307
Epoch 3/10
2180/2180 [==============================] - 20s 9ms/step - loss: 5.9519 - acc: 0.6307
Epoch 4/10
2180/2180 [==============================] - 21s 9ms/step - loss: 5.9519 - acc: 0.6307
Epoch 5/10
2180/2180 [==============================] - 21s 9ms/step - loss: 5.9519 - acc: 0.6307
Epoch 6/10
2180/2180 [==============================] - 20s 9ms/step - loss: 5.9519 - acc: 0.6307
Epoch 7/10
2180/2180 [==============================] - 21s 9ms/step - loss: 5.9519 - acc: 0.6307
Epoch 8/10
2180/2180 [==============================] - 21s 10ms/step - loss: 5.9519 - acc: 0.6307
Epoch 9/10
2180/2180 [==============================] - 21s 10ms/step - loss: 5.9519 - acc: 0.6307
Epoch 10/10
2180/2180 [==============================] - 21s 9ms/step - loss: 5.9519 - acc: 0.6307


In [17]:
Y_pred = conv_base.predict(X_test_normalize)
Y_pred

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [18]:
ans = [np.argmax(i) for i in Y_pred]
print(ans.count(0), ans.count(1), ans.count(2),ans.count(3))

524 0 0 0
